# Sensors

In [47]:
import os
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

import time
import glob
import tables as tb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import numpy.testing as npt
from operator import itemgetter, attrgetter
%matplotlib inline
%load_ext autoreload
%autoreload 2

plt.rcParams["figure.figsize"] = 10, 8
plt.rcParams["font.size"     ] = 14

2017-12-22 12:51:39
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
from math import sqrt, pi

In [61]:
from pynext.system_of_units import *
from pynext.Sensors import PMT
from pynext.Sensors import SiPM
from pynext.Sensors import KDB

In [62]:
from collections import namedtuple

In [63]:
from pynext.activity_functions import Activity
from pynext.activity_functions import activity_table
from pynext.activity_functions import pmt_activity
from pynext.activity_functions import sipm_activity
from pynext.activity_functions import print_activity

In [64]:
from pynext.CylindricalVessel import CylindricalVessel
from pynext.CylindricalVessel import CVD, CVA
from pynext.NextData import NextPVData
from pynext.Material import vacuum, ti316, cu12, cu03, pb

## PMTs

In [65]:
pmt = PMT()
pmt


        PMT name                = R1141
        PMT Diameter            =    7.50 cm
        PMT Surface             =   44.18 cm2
        PMT QE                  =    0.30
        total activity:
        bi214 activity          =    1.09 mBq
        tl208 activity          =    0.45 mBq

        partial activities
        bi214 activity (PMT)    =    0.17 mBq
        tl208 activity (PMT)    =    0.10 mBq
        bi214 activity (Base)   =    0.61 mBq
        tl208 activity (Base)   =    0.17 mBq
        bi214 activity (Window) =    0.30 mBq
        tl208 activity (Window) =    0.19 mBq
        

## SiPMs

In [66]:
sipm = SiPM()
sipm


        SiPM name          = SENSL
        SiPM size          =    1.00 mm
        SiPM Surface       =    1.00 mm2
        SiPM QE            =    0.50
        SiPM TPB eff       =    0.50
        SiPM global PDE    =    0.25
        bi214 activity     =    0.05 muBq
        bi214 activity     =    0.05 muBq
        

## KDBs

In [57]:
kdb = KDB(L = 110  * mm,
          pitch    = 10   * mm,
          nof_sipm = 64,
          a_bi214  = 31 * muBq, a_tl208= 15 * muBq)
kdb


        nof_sipm          = 64
        KDB size          =  110.00 mm
        KDB pitch         =   10.00 mm
        KDB Surface       =  121.00 cm2
        bi214 activity    =   15.50 muBq
        bi214 activity    =    7.50 muBq
        

In [58]:
nof_kdb = n100_pv.head_surface / kdb.S
nof_kdb 

120.0555738032992

In [59]:
nof_sipm = nof_kdb * kdb.nof_sipm
nof_sipm

7683.556723411149

In [67]:
pmtActivity = pmt_activity('PMT activity', 60, pmt)

In [68]:
print_activity('PMT activity', pmtActivity, unit='mBq')


    activity 	 PMT activity
    Bi-214 	   16.48 mBq
    Tl-208 	    7.86 mBq
    


In [69]:
sipmActivity = sipm_activity('SiPM activity', nof_sipm, sipm)

In [70]:
print_activity('SiPM activity', sipmActivity, unit='mBq')


    activity 	 SiPM activity
    Bi-214 	    0.19 mBq
    Tl-208 	    0.19 mBq
    


In [71]:
kdbActivity = sipm_activity('KDB activity', nof_kdb , kdb)
print_activity('KDB activity', kdbActivity, unit='mBq')


    activity 	 KDB activity
    Bi-214 	    0.93 mBq
    Tl-208 	    0.45 mBq
    


In [72]:
act_sensor_table =activity_table([pmtActivity, sipmActivity, kdbActivity])
print(act_sensor_table)

            name      bi214     tl208
0   PMT activity  16.477500  7.860000
1  SiPM activity   0.192089  0.192089
2   KDB activity   0.930431  0.450208


In [73]:
act_sensor_table

,name,bi214,tl208
0,PMT activity,16.477500,7.860000
1,SiPM activity,0.192089,0.192089
2,KDB activity,0.930431,0.450208
